# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [3]:
# Check the shape of the data
print("Shape of the data:", spaceship.shape)


Shape of the data: (8693, 14)


**Check for data types**

In [4]:
# Check data types
print("Data types of each column:")
print(spaceship.dtypes)


Data types of each column:
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object


**Check for missing values**

In [5]:
# Check for missing values
print("Missing values in each column:")
print(spaceship.isnull().sum())


Missing values in each column:
PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [6]:
# Drop rows with missing values
spaceship_cleaned = spaceship.dropna()

# Confirm no missing values remain
print("Missing values after cleaning:")
print(spaceship_cleaned.isnull().sum())


Missing values after cleaning:
PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64


- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [7]:
# Transform Cabin to keep only the first letter
spaceship_cleaned['Cabin'] = spaceship_cleaned['Cabin'].str[0]

# Verify unique values in Cabin
print("Unique values in Cabin after transformation:")
print(spaceship_cleaned['Cabin'].unique())


Unique values in Cabin after transformation:
['B' 'F' 'A' 'G' 'E' 'C' 'D' 'T']


c:\Users\Andre\Python 3.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


- Drop PassengerId and Name

In [8]:
# Drop PassengerId and Name
spaceship_cleaned = spaceship_cleaned.drop(['PassengerId', 'Name'], axis=1)

# Confirm the columns are removed
print("Columns after dropping PassengerId and Name:")
print(spaceship_cleaned.columns)


Columns after dropping PassengerId and Name:
Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported'],
      dtype='object')


- For non-numerical columns, do dummies.

In [9]:
# Convert non-numerical columns to dummy variables
spaceship_encoded = pd.get_dummies(spaceship_cleaned, drop_first=True)

# Verify the new DataFrame structure
print("Data after encoding categorical variables:")
print(spaceship_encoded.head())


Data after encoding categorical variables:
    Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  Transported  \
0  39.0          0.0        0.0           0.0     0.0     0.0        False   
1  24.0        109.0        9.0          25.0   549.0    44.0         True   
2  58.0         43.0     3576.0           0.0  6715.0    49.0        False   
3  33.0          0.0     1283.0         371.0  3329.0   193.0        False   
4  16.0        303.0       70.0         151.0   565.0     2.0         True   

   HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  Cabin_B  Cabin_C  \
0                  1                0               0        1        0   
1                  0                0               0        0        0   
2                  1                0               0        0        0   
3                  1                0               0        0        0   
4                  0                0               0        0        0   

   Cabin_D  Cabin_E  Cabin_F  Cabin_G

**Perform Train Test Split**

In [10]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = spaceship_encoded.drop('Transported', axis=1)  # Features (sin la columna objetivo)
y = spaceship_encoded['Transported']  # Target (columna objetivo)

# Split the data into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar la división
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (5284, 19)
Shape of X_test: (1322, 19)
Shape of y_train: (5284,)
Shape of y_test: (1322,)


**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [11]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize KNN
knn = KNeighborsClassifier()

# Train the model using the training data
knn.fit(X_train, y_train)

print("KNN model has been trained.")


KNN model has been trained.


- Evaluate your model's performance. Comment it

In [12]:
# Evaluate the model
accuracy = knn.score(X_test, y_test)
print("Accuracy of the KNN model:", accuracy)


Accuracy of the KNN model: 0.7798789712556732
